# Importing libraries

In [1]:
#!/usr/bin/env python

# testing face alignment, as on learnopencv.com
# cd Desktop/CMU/"Face Alignment"/FaceAverage

#Amogh description of this file: used to debug changes in the landmarks and the faces. To be run in Python 2.7 environment. 
import os
import cv2
import numpy as np
import math
import sys

# Copying landmark files to calculate the average landmarks

Code below in raw to avoid copying again

'E:/CMU/approach1_april_10/data/CK+/cohn-kanade-images/' now has landmark files and png files as well

# Defining helper functions for aligning images

In [2]:
# Read points from text files in directory
def readPoints(path) :
    print 'trying to read points from ', path
    # Create an array of array of points.
    pointsArray = [];

    #List all files in the directory and read points from text files one by one
#     for filePath in os.listdir(path):
        
#         if filePath.endswith("Image-20.jpg.txt"):
            
            #Create an array of points.
    points = [];            
            
            # Read points from filePath
    with open(path) as file :
        for line in file :
            x, y = line.split()
            points.append((int(float(x)), int(float(y))))

    # Store array of points
    pointsArray.append(points)
            
    return pointsArray;

# Read all jpg images in folder.
def readImages(path) :
    print 'trying to read points from: ', path
    #Create array of array of images.
    imagesArray = [];
    
    #List all files in the directory and read points from text files one by one
#     for filePath in os.listdir(path):
#         if filePath.endswith("Image-20.jpg"):
            # Read image found.
    img = cv2.imread(path);

    # Convert to floating point
    img = np.float32(img)/255.0;

    # Add to array of images
    imagesArray.append(img);
            
    return imagesArray;
                
# Compute similarity transform given two sets of two points.
# OpenCV requires 3 pairs of corresponding points.
# We are faking the third one.

def similarityTransform(inPoints, outPoints) :
    s60 = math.sin(60*math.pi/180);
    c60 = math.cos(60*math.pi/180);  
  
    inPts = np.copy(inPoints).tolist();
    outPts = np.copy(outPoints).tolist();
    
    xin = c60*(inPts[0][0] - inPts[1][0]) - s60*(inPts[0][1] - inPts[1][1]) + inPts[1][0];
    yin = s60*(inPts[0][0] - inPts[1][0]) + c60*(inPts[0][1] - inPts[1][1]) + inPts[1][1];
    
    inPts.append([np.int(xin), np.int(yin)]);
    
    xout = c60*(outPts[0][0] - outPts[1][0]) - s60*(outPts[0][1] - outPts[1][1]) + outPts[1][0];
    yout = s60*(outPts[0][0] - outPts[1][0]) + c60*(outPts[0][1] - outPts[1][1]) + outPts[1][1];
    
    outPts.append([np.int(xout), np.int(yout)]);
    
    tform = cv2.estimateRigidTransform(np.array([inPts]), np.array([outPts]), False);
    
    return tform;


# Check if a point is inside a rectangle
def rectContains(rect, point) :
    if point[0] < rect[0] :
        return False
    elif point[1] < rect[1] :
        return False
    elif point[0] > rect[2] :
        return False
    elif point[1] > rect[3] :
        return False
    return True

# Calculate delanauy triangle
def calculateDelaunayTriangles(rect, points):
    # Create subdiv
    subdiv = cv2.Subdiv2D(rect);
   
    # Insert points into subdiv
    for p in points:
            subdiv.insert((p[0], p[1]));

   
    # List of triangles. Each triangle is a list of 3 points ( 6 numbers )
    triangleList = subdiv.getTriangleList();

    # Find the indices of triangles in the points array

    delaunayTri = []
    
    for t in triangleList:
        pt = []
        pt.append((t[0], t[1]))
        pt.append((t[2], t[3]))
        pt.append((t[4], t[5]))
        
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])        
        
        if rectContains(rect, pt1) and rectContains(rect, pt2) and rectContains(rect, pt3):
            ind = []
            for j in xrange(0, 3):
                for k in xrange(0, len(points)):                    
                    if(abs(pt[j][0] - points[k][0]) < 1.0 and abs(pt[j][1] - points[k][1]) < 1.0):
                        ind.append(k)                            
            if len(ind) == 3:                                                
                delaunayTri.append((ind[0], ind[1], ind[2]))
        

    
    return delaunayTri


def constrainPoint(p, w, h) :
    p =  ( min( max( p[0], 0 ) , w - 1 ) , min( max( p[1], 0 ) , h - 1 ) )
    return p;

# Apply affine transform calculated using srcTri and dstTri to src and
# output an image of size.
def applyAffineTransform(src, srcTri, dstTri, size) :
    
    # Given a pair of triangles, find the affine transform.
    warpMat = cv2.getAffineTransform( np.float32(srcTri), np.float32(dstTri) )
    
    # Apply the Affine Transform just found to the src image
    dst = cv2.warpAffine( src, warpMat, (size[0], size[1]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101 )

    return dst


# Warps and alpha blends triangular regions from img1 and img2 to img
def warpTriangle(img1, img2, t1, t2) :

    # Find bounding rectangle for each triangle
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))

    # Offset points by left top corner of the respective rectangles
    t1Rect = [] 
    t2Rect = []
    t2RectInt = []

    for i in xrange(0, 3):
        t1Rect.append(((t1[i][0] - r1[0]),(t1[i][1] - r1[1])))
        t2Rect.append(((t2[i][0] - r2[0]),(t2[i][1] - r2[1])))
        t2RectInt.append(((t2[i][0] - r2[0]),(t2[i][1] - r2[1])))


    # Get mask by filling triangle
    mask = np.zeros((r2[3], r2[2], 3), dtype = np.float32)
    cv2.fillConvexPoly(mask, np.int32(t2RectInt), (1.0, 1.0, 1.0), 16, 0);

    # Apply warpImage to small rectangular patches
    img1Rect = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]
    
    size = (r2[2], r2[3])

    img2Rect = applyAffineTransform(img1Rect, t1Rect, t2Rect, size)
    
    img2Rect = img2Rect * mask

    # Copy triangular region of the rectangular patch to the output image
    img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] = img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] * ( (1.0, 1.0, 1.0) - mask )
     
    img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] = img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] + img2Rect

# Function to alignmage

In [3]:
# path = 'C:/Users/AMOGH GUPTA/Desktop/CMU/Face Alignment/FaceAverage/try_faces/'
# files_image=
# to run the function, enter the absolute path of the image 
def alignAndcrop(img_abs_path):
    # Dimensions of output image
    w = 112;
    h = 112;
    landmark_path=img_abs_path.split('.')[0]+'_landmarks.txt'
    # Read points for all images
    allPoints = readPoints(landmark_path);
    
    # Read all images
    images = readImages(img_abs_path);
    
    # Eye corners
    eyecornerDst = [ (np.int(0.3 * w ), np.int(h / 3)), (np.int(0.7 * w ), np.int(h / 3)) ];
    
    imagesNorm = [];
    pointsNorm = [];
    
    # Add boundary points for delaunay triangulation
    boundaryPts = np.array([(0,0), (w/2,0), (w-1,0), (w-1,h/2), ( w-1, h-1 ), ( w/2, h-1 ), (0, h-1), (0,h/2) ]);
    
    # Initialize location of average points to 0s
    pointsAvg = np.array([(0,0)]* ( len(allPoints[0]) + len(boundaryPts) ), np.float32());
    
    n = len(allPoints[0]);

    numImages = len(images)
    
    # Warp images and trasnform landmarks to output coordinate system,
    # and find average of transformed landmarks.
    
    for i in xrange(0, numImages):

        points1 = allPoints[i];

        # Corners of the eye in input image
        eyecornerSrc  = [ allPoints[i][36], allPoints[i][45] ] ;
        
        # Compute similarity transform
        tform = similarityTransform(eyecornerSrc, eyecornerDst);
        
        # Apply similarity transformation
        img = cv2.warpAffine(images[i], tform, (w,h));

        # Apply similarity transform on points
        points2 = np.reshape(np.array(points1), (68,1,2));        
        points = cv2.transform(points2, tform);
        points = np.float32(np.reshape(points, (68, 2)));
        
        # Append boundary points. Will be used in Delaunay Triangulation
        points = np.append(points, boundaryPts, axis=0)
        
        # Calculate location of average landmark points.
        pointsAvg = pointsAvg + points / numImages;
        
        pointsNorm.append(points);
        imagesNorm.append(img);
        # cv2.imshow('image1', img);
#amogh debug - trying to save the average landmarks. pc format is the one like learnopencv tutorial, and the android format is the one with  commas to be able tp save in an array.
    # np.savetxt('txt_landmarks/androidformat.txt', pointsAvg,fmt= '%1.3f', newline=',',delimiter=',')
    # np.savetxt('txt_landmarks/pcformat.txt', pointsAvg,fmt= '%1.3f',delimiter=' ')
    # print pointsAvg

    #redefining average points
    pointsAvg = [];            
    with open('C:/Users/AMOGH GUPTA/Desktop/CMU/Face Alignment/FaceAverage/txt_landmarks/pcformat.txt') as file :
        for line in file :
            x, y = line.split()
            pointsAvg.append([float(x), float(y)])
    # print pointsAvg
    
    rect = (0, 0, w, h);
    dt = calculateDelaunayTriangles(rect, np.array(pointsAvg));

    # Output image
    output = np.zeros((h,w,3), np.float32());

    # Warp input images to average image landmarks
    for i in xrange(0, len(imagesNorm)) :
        img = np.zeros((h,w,3), np.float32());
        # Transform triangles one by one
        for j in xrange(0, len(dt)) :
            tin = []; 
            tout = [];
            
            for k in xrange(0, 3) :                
                pIn = pointsNorm[i][dt[j][k]];
                pIn = constrainPoint(pIn, w, h);
                
                pOut = pointsAvg[dt[j][k]];
                pOut = constrainPoint(pOut, w, h);
                
                tin.append(pIn);
                tout.append(pOut);
            
            
            warpTriangle(imagesNorm[i], img, tin, tout);

        cv2.imshow('image3', img);

        # Add image intensities for averaging
        output = output + img;
    # Divide by numImages to get average
    output = output / numImages;
    mask = np.zeros((output.shape[0], output.shape[1], 3), dtype = np.float32)
    # print len(points)
    print np.flip(points[2:7],axis=0)
    # print(np.concatenate((points[0:4],points[5:8:-1]),axis=0))
    landmarks_to_mask=np.concatenate((points[0:17],np.flip(points[17:27],axis=0)),axis=0)
    cv2.fillConvexPoly(mask, np.int32(landmarks_to_mask), (1.0, 1.0, 1.0), 16, 0);
    output=output*mask
    # hog.compute(src,descriptors);
    # cv2.circle(output,(10,10),2,(0,0,255),2)
    # for p in pointsAvg:
        # int p1=(int)p[0]
        # int p2=(int)p[1]
        # cv2.circle(output, (int(p[0]),int(p[1])), 1, (0,0,255), 1)
    for p in points[:27]:
        cv2.circle(output, (int(p[0]),int(p[1])), 1, (0,255,0), 1)
    # Display result
    cv2.imwrite(('E:/CMU/approach1_april_10/data/cropped/'+img_abs_path.split('/')[-1]),output)
    cv2.imshow('image', output);
    cv2.waitKey(4000);

In [4]:
# Action Plan
 

# Loading CK+ dataset

First modified the CK+ dataset by using a a script from https://github.com/ifp-uiuc/do-neural-networks-learn-faus-iccvw-2015/tree/master/data_scripts

#### Loading FAUs according to file name in a dictionary

In [5]:
import glob
import os
import sys

##### Paths

In [6]:
ck_modified_image_path='E:/CMU/approach1_april_10/data/cohn-kanade-images/'
ck_modified_fau_label_path='E:/CMU/approach1_april_10/data/FACS/'
ck_original_path='E:/CMU/approach1_april_10/data/CK+/cohn-kanade-images/'

##### Make a dictionary of the dataset {subj/seq : fau_dictionary}
###### This cell populates dict_file_fau in such a way -> {'S129/006': {17.0: 2.0, 23.0: 4.0, 6.0: 2.0, 7.0: 4.0}, 'S129/007': {1.0: 2.0, 2.0: ....}
###### also populates dict_subjects_sequence with{'S005': ['001'], 'S010': ['001', '002', '003', '004', '005', '006'], 'S011': ['001', '002', '003', '004', '005', '006'],

In [7]:
subjects_list=[subj for subj in os.listdir(ck_modified_image_path)]
dict_subjects_sequence={}

dict_file_FAU={}

facs_txt_files_list=[]

for subj in subjects_list:
#     print "the subject is: ", subj
    dict_subjects_sequence[subj]=[]
    sequence_path=ck_modified_fau_label_path+'/'+subj
    sequence_list=os.listdir(sequence_path)
#     print sequence_list
    for seq in sequence_list:
        dict_subjects_sequence[subj].append(seq)
#         print seq # output eg-  001
        file_path=sequence_path+'/'+seq+'/'
#         print file_path #output eg- E:/CMU/approach1_april_10/data/FACS//S005/001/
        txt_file_path=file_path+os.listdir(file_path)[0]
#         print txt_file_path#output eg- E:/CMU/approach1_april_10/data/FACS//S005/001/S005_001_00000011_facs.txt
        facs_txt_files_list.append(txt_file_path)
        txt_file=open(txt_file_path,'r')
        txt=txt_file.readlines()
        dict_fau={}
        for line in txt:
            ln=line.split()
            if len(ln)>1:
                dict_fau[float(ln[0])]=float(ln[1])
        dict_file_FAU[subj+'/'+seq]=dict_fau
#         print glob.glob(file_path+'/*.*')[0]
print 'dict_file_FAU is populated and has these many elements: ', (len(dict_file_FAU.keys()))
print 'facs_txt_files_list is populated and has these many elements: ', len(facs_txt_files_list)
# This cell populates dict_file_fau in such a way -> {'S129/006': {17.0: 2.0, 23.0: 4.0, 6.0: 2.0, 7.0: 4.0}, 'S129/007': {1.0: 2.0, 2.0: ....}

dict_file_FAU is populated and has these many elements:  571
facs_txt_files_list is populated and has these many elements:  571


In [8]:
facs_txt_files_list[0].split('/')

['E:',
 'CMU',
 'approach1_april_10',
 'data',
 'FACS',
 '',
 'S005',
 '001',
 'S005_001_00000011_facs.txt']

##### This cell populates image_file_list

In [9]:
image_file_list=[]
for subj in os.listdir(ck_modified_image_path):
    sequence_path=ck_modified_image_path+'/'+subj
    for seq in os.listdir(sequence_path):
        seq_im_path=sequence_path+'/'+seq
        if (subj in dict_subjects_sequence.keys()) and (seq in dict_subjects_sequence[subj]):
            image_file_list.append(os.listdir(seq_im_path)[-1])
print 'image_file_list has been populated and has these many png files: ', len(image_file_list)           
# this cell populates image_file_list in such a way -> []'S005_001_00000011.png', 'S010_002_00000014.png']

image_file_list has been populated and has these many png files:  327


In [10]:
image_abs_file_list=[]
for f in image_file_list:
    part=f.split('_')
    image_abs_file_list.append(ck_original_path+part[0]+'/'+part[1]+'/'+f)
print 'image_abs_file_list has been populated and has these many elements: ', len(image_abs_file_list)
# image_abs_file_list=[[part][0] for f in image_file_list for part in f.split('_')]

image_abs_file_list has been populated and has these many elements:  327


In [11]:
alignAndcrop(image_abs_file_list[0])

trying to read points from  E:/CMU/approach1_april_10/data/CK+/cohn-kanade-images/S005/001/S005_001_00000011_landmarks.txt
trying to read points from:  E:/CMU/approach1_april_10/data/CK+/cohn-kanade-images/S005/001/S005_001_00000011.png
[[ 38.  86.]
 [ 31.  81.]
 [ 25.  74.]
 [ 22.  66.]
 [ 18.  57.]]


##### Debugging function to check in which subj/seq is a particular FAU present

In [11]:
i=0
j=0
for k,v in dict_file_FAU.iteritems():
    FAU_id=2.0
    FAU_threshold=1.0
    if FAU_id in v.keys():
            i+=1
            if (v[FAU_id]>=FAU_threshold):
                j+=1
                print k, ' has FAU ', FAU_id ,' value as ', v[FAU_id]           
print "This FAU is present in ", i
print "It is equal or above threshold in ", j

S129/007  has FAU  2.0  value as  2.0
S080/005  has FAU  2.0  value as  1.0
S506/006  has FAU  2.0  value as  2.0
S506/004  has FAU  2.0  value as  4.0
S138/007  has FAU  2.0  value as  2.0
S090/002  has FAU  2.0  value as  4.0
S160/006  has FAU  2.0  value as  2.0
S504/004  has FAU  2.0  value as  2.0
S504/006  has FAU  2.0  value as  1.0
S085/003  has FAU  2.0  value as  3.0
S056/003  has FAU  2.0  value as  4.0
S999/003  has FAU  2.0  value as  1.0
S060/003  has FAU  2.0  value as  4.0
S055/006  has FAU  2.0  value as  1.0
S051/002  has FAU  2.0  value as  4.0
S132/008  has FAU  2.0  value as  3.0
S125/007  has FAU  2.0  value as  3.0
S126/004  has FAU  2.0  value as  4.0
S125/006  has FAU  2.0  value as  3.0
S054/003  has FAU  2.0  value as  2.0
S068/003  has FAU  2.0  value as  3.0
S133/009  has FAU  2.0  value as  1.0
This FAU is present in  116
It is equal or above threshold in  22


## Relevant FAU to tutoring
#### 1 inner brow raiser

#### 2 outer brow raiser

#### 4 brow lowerer

#### 5 upper lid raiser

#### 7 lid tightener

#### 25 lips part

#### 26 jaw drop

In [ ]:
path = 'C:/Users/AMOGH GUPTA/Desktop/CMU/Face Alignment/FaceAverage/try_faces/'
    
    # Dimensions of output image
    w = 112;
    h = 112;

    # Read points for all images
    allPoints = readPoints(path);
    
    # Read all images
    images = readImages(path);
    
    # Eye corners
    eyecornerDst = [ (np.int(0.3 * w ), np.int(h / 3)), (np.int(0.7 * w ), np.int(h / 3)) ];
    
    imagesNorm = [];
    pointsNorm = [];
    
    # Add boundary points for delaunay triangulation
    boundaryPts = np.array([(0,0), (w/2,0), (w-1,0), (w-1,h/2), ( w-1, h-1 ), ( w/2, h-1 ), (0, h-1), (0,h/2) ]);
    
    # Initialize location of average points to 0s
    pointsAvg = np.array([(0,0)]* ( len(allPoints[0]) + len(boundaryPts) ), np.float32());
    
    n = len(allPoints[0]);

    numImages = len(images)
    
    # Warp images and trasnform landmarks to output coordinate system,
    # and find average of transformed landmarks.
    
    for i in xrange(0, numImages):

        points1 = allPoints[i];

        # Corners of the eye in input image
        eyecornerSrc  = [ allPoints[i][36], allPoints[i][45] ] ;
        
        # Compute similarity transform
        tform = similarityTransform(eyecornerSrc, eyecornerDst);
        
        # Apply similarity transformation
        img = cv2.warpAffine(images[i], tform, (w,h));

        # Apply similarity transform on points
        points2 = np.reshape(np.array(points1), (68,1,2));        
        points = cv2.transform(points2, tform);
        points = np.float32(np.reshape(points, (68, 2)));
        
        # Append boundary points. Will be used in Delaunay Triangulation
        points = np.append(points, boundaryPts, axis=0)
        
        # Calculate location of average landmark points.
        pointsAvg = pointsAvg + points / numImages;
        
        pointsNorm.append(points);
        imagesNorm.append(img);
        # cv2.imshow('image1', img);
#amogh debug - trying to save the average landmarks. pc format is the one like learnopencv tutorial, and the android format is the one with  commas to be able tp save in an array.
    # np.savetxt('txt_landmarks/androidformat.txt', pointsAvg,fmt= '%1.3f', newline=',',delimiter=',')
    # np.savetxt('txt_landmarks/pcformat.txt', pointsAvg,fmt= '%1.3f',delimiter=' ')
    # print pointsAvg

    #redefining average points
    pointsAvg = [];            
    with open('C:/Users/AMOGH GUPTA/Desktop/CMU/Face Alignment/FaceAverage/txt_landmarks/pcformat.txt') as file :
        for line in file :
            x, y = line.split()
            pointsAvg.append([float(x), float(y)])
    # print pointsAvg
    
    rect = (0, 0, w, h);
    dt = calculateDelaunayTriangles(rect, np.array(pointsAvg));

    # Output image
    output = np.zeros((h,w,3), np.float32());

    # Warp input images to average image landmarks
    for i in xrange(0, len(imagesNorm)) :
        img = np.zeros((h,w,3), np.float32());
        # Transform triangles one by one
        for j in xrange(0, len(dt)) :
            tin = []; 
            tout = [];
            
            for k in xrange(0, 3) :                
                pIn = pointsNorm[i][dt[j][k]];
                pIn = constrainPoint(pIn, w, h);
                
                pOut = pointsAvg[dt[j][k]];
                pOut = constrainPoint(pOut, w, h);
                
                tin.append(pIn);
                tout.append(pOut);
            
            
            warpTriangle(imagesNorm[i], img, tin, tout);

        cv2.imshow('image3', img);

        # Add image intensities for averaging
        output = output + img;

    output = output / numImages;
    mask = np.zeros((output.shape[0], output.shape[1], 3), dtype = np.float32)
    print np.flip(points[2:7],axis=0)
    landmarks_to_mask=np.concatenate((points[0:17],np.flip(points[17:27],axis=0)),axis=0)
    cv2.fillConvexPoly(mask, np.int32(landmarks_to_mask), (1.0, 1.0, 1.0), 16, 0);
    output=output*mask

In [52]:
int(float('2.00000e+02'))

200